# Visualising the traffic around Boston Airport

Aim of this notebook is to provide a codealong to be able to recreate the traffic from Square One Mall in Saugus, Massachusetts to Boston Logan Airport.

In [1]:
# Importing the necessary packages:
import pandas as pd
import numpy as np
import json
import requests

In [2]:
#Retrieving my api keys information to access the Google API.
def get_keys(path):
    with open(path) as f:
        return json.load(f)
    
keys = get_keys("/Users/jjherranzsarrion/.secret/google_blog2_api.json")

api_key = keys['api_key']

url = 'https://maps.googleapis.com/maps/api/directions/json?'

origin = '323+Broadway+Saugus+MA+01906'
destination = 'Terminal+C+Boston+Logan+International+Airport+Boston+MA+02128'
departure_time = '1566819000' #time in seconds from midnight 1st Jan 1970 (Unix start time) until Monday 19th August at 07:30 AM. 
 
url_params = f"origin={origin}&destination={destination}&departure_time={departure_time}&key={api_key}"

request_url = url + url_params
    
print(request_url)    
    
response = requests.get(request_url)
print(type(response.text))
print(response.text)

with open('response.json', 'w') as f:
    json.dump(response.json(), f)

In [21]:
with open('response.json', 'r') as f:
    directions = json.load(f)

In [22]:
directions.keys()

dict_keys(['geocoded_waypoints', 'routes', 'status'])

In [24]:
route = directions['routes']

In [6]:
root_steps = []
for legs in route:
    for steps in legs['legs']:
        for coordinates in steps['steps']:
            root_steps.append(coordinates['start_location'])

#Chosen to only select the start coordinate of each step given that the end coordinate of step 0 
#is the start coordinate of step 1. The last coordinate is added at the end of the loop.

In [7]:
# Unpacking the root_steps list to retrieve only latitude and longitude data. Adding weights to represent different
# traffic congestions at each step of the root.

weighted_coordinates = [(root_step['lat'], root_step['lng'], np.random.randint(1,4) ) for root_step in root_steps]

In [8]:
weighted_coordinates_df = pd.DataFrame(weighted_coordinates, columns=['latitude', 'longitude', 'weight'])

In [9]:
weighted_coordinates_df

,latitude,longitude,weight
0,42.485146,-70.960539,1
1,42.473452,-70.976922,3
2,42.471169,-70.978031,2
3,42.468867,-70.976501,1
4,42.463469,-70.986506,3
5,42.461928,-70.986496,2
6,42.460390,-70.988740,2
7,42.455216,-70.990396,3
8,42.449546,-70.980647,1
9,42.421689,-71.004522,3


In [25]:
import gmaps
import gmaps.datasets
%matplotlib inline

gmaps.configure(api_key=api_key)

locations = weighted_coordinates_df[['latitude', 'longitude']]
weights = weighted_coordinates_df['weight']

fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
m = gmaps.Map()
heatmap_layer = gmaps.Heatmap(data=weighted_coordinates)
m.add_layer(heatmap_layer)
m